In [17]:
import pandas as pd
import numpy as np
import pickle
import time
import requests
import datetime
from pandas import json_normalize

In [18]:
df_merged = pd.read_csv('df_merged.csv')
df_merged

,Unnamed: 0,stock,Bearish,Neutral,Bullish,Total_Compound,Date,high52,low52,returnOnEquity,returnOnAssets,totalDebtToCapital,totalDebtToEquity,vol10DayAvg,pegRatio,marketCap,dividendYield,epsTTM
0,0,TSLA,0.146714,0.692571,0.160679,-0.065818,1645506000,1243.4900,546.9800,21.07834,9.87758,17.78900,22.63738,24059852.0,0.307284,1080801.000,0.00,4.92162
1,1,DKNG,0.035870,0.779870,0.184217,0.495400,1645506000,64.6000,14.9700,0.00000,0.00000,42.65325,74.37779,19558468.0,0.000000,7225.412,0.00,0.00000
2,2,FB,0.130636,0.692909,0.176364,0.093436,1645506000,384.3300,185.8200,31.10175,24.20513,0.46310,0.46525,28533150.0,0.441493,607809.600,0.00,13.78663
3,3,DWAC,0.147533,0.752067,0.100267,-0.025420,1645506000,175.0000,9.8401,0.00000,0.00000,0.00000,0.00000,2665709.0,0.000000,1767.700,0.00,0.00000
4,4,AMD,0.034583,0.669250,0.296333,0.491358,1645506000,164.4599,72.5000,47.42763,29.52154,4.00768,4.17500,103828029.0,1.540659,167961.800,0.00,2.57396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,185,AMD,0.114756,0.707111,0.178100,0.106509,1648699200,164.4599,72.5000,47.42763,29.52154,4.00768,4.17500,103828029.0,1.540659,167961.800,0.00,2.57396
186,186,TSLA,0.089564,0.774218,0.136200,0.115873,1648699200,1243.4900,546.9800,21.07834,9.87758,17.78900,22.63738,24059852.0,0.307284,1080801.000,0.00,4.92162
187,187,TLRY,0.132000,0.671821,0.196214,0.099411,1648699200,23.0400,4.7800,0.00000,0.00000,13.21841,15.34299,103438612.0,0.000000,3485.251,0.00,0.00000
188,188,NVDA,0.090500,0.719833,0.189722,0.296533,1648699200,346.4700,134.5900,44.83623,26.72860,29.14426,41.13182,54131912.0,0.514619,612615.700,0.07,3.84519


In [21]:
# def clean_merged_df(df_merged):
    
#     saturday_rows=[]
    
#     x = list(df_merged["Date"])
#     counter1=0 #Count of row in data_frame
#     for date in df:
#         dotw = ((date//86400)+4)%7
#         if (dotw==0):
#             saturday_rows.append(df_merged.index[counter1])
#         counter1+=1
    
#     df_merged_new = df_merged.drop(saturday_rows)
#     return df_merged_new
            
# #     check if all the columns were removed.

In [19]:
#Getting rid of satur-days

df_merged.drop(df_merged.index[(((df_merged['Date'] //86400)+4)%7)==6], inplace=True)
df_merged=df_merged.reset_index()

In [20]:
# #Loading and cleaning df_merged

# df_merged = pd.read_csv('df_merged.csv')
# df_merged = clean_merged_df(df_merged)
# df_merged

In [22]:
#Producing the stock list

STOCKS = list(df_merged['stock'])

#Producing the date list

DATE = list(df_merged['Date'])

#Initiallizing the price_df
bad_api_calls = []
price_df = pd.DataFrame()



STOCKS

['TSLA',
 'DKNG',
 'FB',
 'DWAC',
 'AMD',
 'TSLA',
 'FB',
 'SP',
 'AMD',
 'CLOV',
 'TSLA',
 'FB',
 'AMD',
 'COIN',
 'TA',
 'TSLA',
 'AMD',
 'NVDA',
 'FB',
 'WISH',
 'CLOV',
 'SP',
 'MAR',
 'TSLA',
 'FB',
 'TSLA',
 'LCID',
 'ZM',
 'SOFI',
 'MSFT',
 'SOFI',
 'TSLA',
 'INTC',
 'AMD',
 'FB',
 'TSLA',
 'SOFI',
 'AMD',
 'FB',
 'AAPL',
 'TSLA',
 'COST',
 'AMD',
 'SOFI',
 'SP',
 'TSLA',
 'CDEV',
 'AMD',
 'HOOD',
 'SOFI',
 'BBBY',
 'TSLA',
 'MU',
 'AMD',
 'AAPL',
 'BBBY',
 'TSLA',
 'SP',
 'NVDA',
 'FB',
 'TSLA',
 'CLOV',
 'AAPL',
 'BBBY',
 'AMD',
 'AMZN',
 'BBBY',
 'TSLA',
 'SP',
 'DKNG',
 'RIVN',
 'AMZN',
 'TSLA',
 'DOCU',
 'CLOV',
 'TSLA',
 'RIVN',
 'AMD',
 'AAPL',
 'SOFI',
 'TSLA',
 'SOFI',
 'AAPL',
 'SP',
 'RIVN',
 'TSLA',
 'SP',
 'AAPL',
 'SOFI',
 'FB',
 'TSLA',
 'ME',
 'SP',
 'HYMC',
 'NVDA',
 'TSLA',
 'SOFI',
 'ME',
 'AMZN',
 'SP',
 'TSLA',
 'AMZN',
 'AMD',
 'TA',
 'NVDA',
 'TSLA',
 'AMD',
 'NVDA',
 'FB',
 'AAPL',
 'MULN',
 'CTXR',
 'BBIG',
 'TSLA',
 'NVDA',
 'TSLA',
 'NVDA',
 'FB',
 'AM

In [23]:
STOCKS

['TSLA',
 'DKNG',
 'FB',
 'DWAC',
 'AMD',
 'TSLA',
 'FB',
 'SP',
 'AMD',
 'CLOV',
 'TSLA',
 'FB',
 'AMD',
 'COIN',
 'TA',
 'TSLA',
 'AMD',
 'NVDA',
 'FB',
 'WISH',
 'CLOV',
 'SP',
 'MAR',
 'TSLA',
 'FB',
 'TSLA',
 'LCID',
 'ZM',
 'SOFI',
 'MSFT',
 'SOFI',
 'TSLA',
 'INTC',
 'AMD',
 'FB',
 'TSLA',
 'SOFI',
 'AMD',
 'FB',
 'AAPL',
 'TSLA',
 'COST',
 'AMD',
 'SOFI',
 'SP',
 'TSLA',
 'CDEV',
 'AMD',
 'HOOD',
 'SOFI',
 'BBBY',
 'TSLA',
 'MU',
 'AMD',
 'AAPL',
 'BBBY',
 'TSLA',
 'SP',
 'NVDA',
 'FB',
 'TSLA',
 'CLOV',
 'AAPL',
 'BBBY',
 'AMD',
 'AMZN',
 'BBBY',
 'TSLA',
 'SP',
 'DKNG',
 'RIVN',
 'AMZN',
 'TSLA',
 'DOCU',
 'CLOV',
 'TSLA',
 'RIVN',
 'AMD',
 'AAPL',
 'SOFI',
 'TSLA',
 'SOFI',
 'AAPL',
 'SP',
 'RIVN',
 'TSLA',
 'SP',
 'AAPL',
 'SOFI',
 'FB',
 'TSLA',
 'ME',
 'SP',
 'HYMC',
 'NVDA',
 'TSLA',
 'SOFI',
 'ME',
 'AMZN',
 'SP',
 'TSLA',
 'AMZN',
 'AMD',
 'TA',
 'NVDA',
 'TSLA',
 'AMD',
 'NVDA',
 'FB',
 'AAPL',
 'MULN',
 'CTXR',
 'BBIG',
 'TSLA',
 'NVDA',
 'TSLA',
 'NVDA',
 'FB',
 'AM

In [24]:
# #Junk to get row

# api_call = get_daily_adjusted_price_data('GME')
# api_call = json_normalize(api_call)
# # dataframe = create_dataframe(api_call)
# # dataframe
# api_call

In [149]:
# # #Junk

# define an endpoint with a stock of your choice, MUST BE UPPER
endpoint = r"https://api.tdameritrade.com/v1/marketdata/{}/pricehistory".format('FB')

# define the payload
payload = {'apikey':'4JHA7B4AE8VS1DQDUGATGHZPQIZNODEO',
           'periodType':'day',
           'frequencyType':'minute',
           'frequency':'1',
           'period':'2',
           'endDate':'1556158524000',
           'startDate':'1554535854000',
           'needExtendedHoursData':'true'}

# make a request
content = requests.get(url = endpoint, params = payload)

print(content)

# convert it dictionary object
data = content.json()

# payload_price = {'apikey': '4JHA7B4AE8VS1DQDUGATGHZPQIZNODEO',
#                            'periodType' : 'day',
#                            'period':'1',
#                            'frequencyType':'daily',
#                            'frequency':'daily',
#                                'endDate':{DATE[0]+86400},
#                                'startDate' : {DATE[0]},
#                                'needExtendedHoursData' : 'false'
#                           }
# api_call = json_normalize(api_call)

data

# if len(api_call['c']) == 1:
#     api_call['c']=api_call['c'][0]
# else:
#     api_call['c']=api_call['c'][0][0]
            
# if len(api_call['o']) == 1:
#     api_call['o'] = api_call['o'][0]
# else:
#     api_call['o']=api_call['o'][0][0]

# if len(api_call['h']) == 1:
#     api_call['h'] = api_call['h'][0]
# else:
#     api_call['h']=api_call['h'][0][0]

# api_call
# #api_call['o']=api_call['o'][0][0]
# api_call['return'] = api_call['o'] - api_call['c']
# if (api_call['return'][0]) > 0:
#     api_call['direction'] = 2
# else:
#     api_call['direction'] = 1

# api_call['stock'] = 'GME'

# # api_call = api_call.drop(columns=['t', 'v', 's', 'l'],axis=1)
# api_call

<Response [200]>


{'candles': [], 'symbol': 'FB', 'empty': True}

In [139]:
#Function to get price and price direction

def create_price_direction_df(stocks,start):
    #CAUTION REMEMBER TO FIX THE START DATE AND END DATE
    df=pd.DataFrame([])
    count=0
    counter_=1
    for stock in stocks:
        print(count)
        endpoint = f"https://api.tdameritrade.com/v1/marketdata/{stock}/pricehistory"
        payload_price = {'apikey': '4JHA7B4AE8VS1DQDUGATGHZPQIZNODEO',
                           'periodType' : 'day',
                           'period':'1',
                           'frequencyType':'daily',
                           'frequency':'daily',
                               'endDate':{start[count]+86400},
                               'startDate' : {start[count]},
                               'needExtendedHoursData' : 'false'
                          }
        
        pricing_content = requests.get(url = endpoint, params=payload_price).json()
        df_temp=pd.DataFrame.from_dict(pricing_content, orient = 'index')
        df = pd.concat([df,df_temp])
        count+=1
        
    return df
        

In [132]:
price_df = create_price_direction_df(STOCKS,DATE)
price_df

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189


,0
error,Bad request.
error,Bad request.
error,Bad request.
error,Bad request.
error,Bad request.
...,...
error,Bad request.
error,Bad request.
error,Bad request.
error,Bad request.


In [133]:
price_df

,0
error,Bad request.
error,Bad request.
error,Bad request.
error,Bad request.
error,Bad request.
...,...
error,Bad request.
error,Bad request.
error,Bad request.
error,Bad request.


In [ ]:
price_df = price_df.reset_index()#temp this CAUTION
price_df

In [ ]:
#Combine the sentiment analysis and price_df and clean
df_merged = pd.concat([df_merged,price_df], axis=1)

#Renaming columns
df_merged = df_merged.rename(columns={'c':'closing_price', 'h':'highest_price', 'o':'opeing_price'})

#Remove duplicated columns
df_final = df_merged.loc[:,~df_merged.columns.duplicated()]
df_final
#Figure how to incorporate specific periods

In [ ]:
#Further cleaning of the final data set

# pd.options.mode.chained_assignment = None  # default='warn' disables warnings.

# #convert to numeric data-type

# int_list = ['bidSize', 'askSize', 'lastSize', 'totalVolume', 'regularMarketLastSize']
# for int_ in int_list:
#     df_final[int_] = pd.to_numeric(df_final[int_])
# df_final['direction'] = pd.to_numeric(df_final['direction'])

# # Columns should be integer data types
# df_final['sharesOutstanding'] = pd.to_numeric(df_final['sharesOutstanding'])
# df_final['vol1DayAvg'] = pd.to_numeric(df_final['vol1DayAvg'])
# df_final['vol10DayAvg'] = pd.to_numeric(df_final['vol10DayAvg'])
# df_final['vol3MonthAvg'] = pd.to_numeric(df_final['vol3MonthAvg'])


# df_final.rename({
# #     '1d-direction': 'direction',
#     'high52': 'highfiftytwo',
#     'low52': 'lowfiftytwo',
#     'vol1DayAvg': 'volonedayavg',
#     'vol10DayAvg': 'voltendayavg',
#     'vol3MonthAvg': 'volthreemonthavg'},
#     axis=1,
#     inplace=True)

df_final.columns = df_final.columns.map(lambda row: "_".join(row.lower().split(" ")))
# df_final = df_final.loc[:, 'direction':]

In [ ]:
#Inputs and outputs into model

X = df_final.drop(['direction','stock','bearish','neutral','bullish','date','closing_price',
 'highest_price','opeing_price','return'], axis=1)
y = df_final['direction']

X
y

# assert int(len(X.columns) +1) == len(df_final.columns)

#df_final_temp = df_final.loc[:, 'direction':]
# df_final_temp
# df_final

In [ ]:
#Functions to encode columns containing categorical data in X

from sklearn.preprocessing import OneHotEncoder

def encode_cats(categoricals, numericals, df):
    """
    Takes in a list of categorical columns and a list of numerical columns and returns the dataframe with encoded variables
    """
    ohe = OneHotEncoder(sparse=False, drop='first')
    cat_matrix = ohe.fit_transform(df.loc[:, categoricals])
    X_ohe = pd.DataFrame(cat_matrix,
                         columns=ohe.get_feature_names(categoricals), #create meaningful column names
                         index=X.index) #keep the same index values
    
    return pd.concat([X.loc[:, numericals], X_ohe], axis=1),ohe

In [ ]:
#Using the above function.

categoricals = list(X.select_dtypes('object').columns)
numericals = list(X.select_dtypes(['int64', 'float64', 'int32', 'float32']).columns)
return_tupple = encode_cats(categoricals, numericals, X)
X = return_tupple[0]
ohe = return_tupple[1]

In [ ]:
categoricals

In [ ]:
(list(X.columns))

In [ ]:
X

In [ ]:
#Actual predicting

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
# import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import auc, roc_auc_score, plot_roc_curve, confusion_matrix, ConfusionMatrixDisplay, precision_recall_curve, PrecisionRecallDisplay, plot_confusion_matrix, precision_score, recall_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

In [ ]:
#Splitting data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42) #hold out 20% of the data for final testing

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)




In [ ]:
#Training

knn = KNeighborsClassifier()
knn.fit(X_train_scaled,y_train)
knn_scores = cross_val_score(knn, X_train_scaled, y_train, cv=5)

rf = RandomForestClassifier()
rf_scores = cross_val_score(rf, X_train_scaled, y_train, cv=5)

print(f"KNN mean scores: {np.mean(knn_scores):.4}")
print(f"Random Forest mean scores: {np.mean(rf_scores):.4}")

len(list(X_train.columns))

In [ ]:
#Pickling/"saving" stuff
import pickle

myVar = {"model": rf, "encoder": ohe}

with open('saved_steps.pkl','wb') as file:
    pickle.dump(myVar,file)

In [ ]:
df_today = pd.read_csv('2022-02-01-2022-02-01.csv')
df_today

stock_list_today = list(df_today['stock'])

In [ ]:
df_today

In [ ]:
####THIS IS THE GET FUNDAMENTAL DATA STUFFF for now(START)

def getFundamental(df,stock_list):
    #Basically get each of the fundamental data points. Concat it

    endpoint_fundamentals = r'https://api.tdameritrade.com/v1/instruments'

    #Defining the payload

    payload_fundamental = {'apikey': '4JHA7B4AE8VS1DQDUGATGHZPQIZNODEO',
                            'symbol': stock_list,
                          'projection':'fundamental'
                          } #We need the tickers seperated by commas.


    fundamental_content = requests.get(url = endpoint_fundamentals, params=payload_fundamental).json()


    fund_cols = []
    for column in [x for x in [*fundamental_content[stock_list[0]]['fundamental']]]:
        fund_cols.append(column)

    # Append data to dataframe
    df_fundamental = pd.DataFrame(columns = fund_cols)
    for stock in stock_list:
        print(stock)
        df_fundamental = df_fundamental.append(pd.Series(fundamental_content[stock]['fundamental'], index=fund_cols), ignore_index=True)


    df_merged = pd.concat([df,df_fundamental], axis=1)
    

    #Removing unecessary columns
    df_merged = df_merged.loc[:, ~df_merged.columns.duplicated()]
    df_merged = df_merged[['stock',
    'Bearish', 'Neutral', 'Bullish', 'Total_Compound', 'Date',
    'high52',
    'low52',
    'returnOnEquity',
    'returnOnAssets',
    'totalDebtToCapital',
    'totalDebtToEquity', 'vol10DayAvg',
    'pegRatio','marketCap','dividendYield','epsTTM']]

    df_merged.drop(df_merged.index[(((df_merged['Date'] //86400)+4)%7)==6], inplace=True) ##GETTING RID OF SATURDAYS
    df_merged=df_merged.reset_index()

    
    #Remove duplicated columns
    df_merged = df_merged.loc[:,~df_merged.columns.duplicated()]
    #df_merged = df_merged.drop(['stock','bearish','neutral','bullish'], axis=1)
    df_merged=df_merged.drop('stock', axis=1)
    df_merged=df_merged.drop('Bearish', axis=1)
    df_merged=df_merged.drop('Neutral', axis=1)
    df_merged=df_merged.drop('Bullish', axis=1)

    
    return df_merged


In [ ]:
X = getFundamental(df_today, stock_list_today)

len(list(X.columns))

In [ ]:
y_pred=knn.predict(X)

In [ ]:
y_pred